In [ ]:
from sec_api import QueryApi, ExtractorApi
from dotenv import load_dotenv
import os

# Initialize SEC API
load_dotenv()
sec_api_key = os.getenv("SEC_API_KEY")
queryApi = QueryApi(api_key=sec_api_key)
extractorApi = ExtractorApi(api_key=sec_api_key)

# List of tickers to process
tickers = ['AAPL', 'MSFT', 'NVDA']

# Make sure folder exists
os.makedirs('mdna_texts', exist_ok=True)

# Get latest 10-K filing for each ticker in the list
for ticker in tickers:
    print(f"\nProcessing {ticker}...")
    try:
        # Query most recent 10-K
        query = {
            "query": {
                "query_string": {
                    "query": f"{ticker} AND formType:\"10-K\""
                }
            },
            "from": "0",
            "size": "1",
            "sort": [{"filedAt": {"order": "desc"}}]
        }
    
        response = queryApi.get_filings(query)
        filing_url = response['filings'][0]['linkToFilingDetails']
        print(f"10-K Filing URL: {filing_url}")

        # Extract MD&A section
        mdna_text = extractorApi.get_section(filing_url, "7")

        # Save to file
        with open(f"mdna_texts/{ticker}_mdna.txt", "w", encoding="utf-8") as f:
            f.write(mdna_text)

        print(f"  Saved MD&A text for {ticker}.\n")
    except Exception as e:
        print(f"  Failed for {ticker}: {e}")

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Initialize Open AI API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=openai_api_key)

# Folder where MD&A texts were saved
mdna_folder = 'mdna_texts'

# Output folder for summaries
os.makedirs("summaries", exist_ok=True)

# Loop through all files in mdna_texts
for filename in os.listdir(mdna_folder):
    if not filename.endswith("_mdna.txt"):
        continue
        
    ticker = filename.split("_")[0]
    print(f"Summarizing MD&A for {ticker}...")

    # Load the text
    with open(os.path.join(mdna_folder, filename), "r", encoding="utf-8") as f:
        mdna_text = f.read()
        
    # Use OpenAI to summarize
    def summarize_financials(text):
        response = openai.chat.completions.create(
            model = 'gpt-3.5-turbo',
            messages = [
                {"role": "system", "content": "You are a financial analyst assistant."},       # tells the model who it is and how to behave
                {"role": "user", "content": f"""Based on the following MD&A section from {ticker}'s latest 10-K filing:  
    {text}

    Please provide the following:
    1. A brief summary (2–3 paragraphs) of the MD&A section.
    2. A list of key investment risks discussed or implied.
    3. A list of potential opportunities or areas of growth.
    4. A full one-page investment research brief, written formally as if to a client.

    Respond in this structure:
    - Summary:
    - Risks:
    - Opportunities:
    - One-Page Brief:
    """}                    # tells the model what to do
            ]
        )
        return response.choices[0].message.content
        
    summary = summarize_financials(mdna_text)
    
    # Save summary to file
    with open(f"summaries/{ticker}_summary.txt", "w", encoding="utf-8") as f:
        f.write(summary)

    print(f"  Saved summary for {ticker}.\n")

In [ ]:
from docx import Document
import os

summaries = 'summaries'
output_folder = 'word_reports'


os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(summaries):
    if not filename.endswith("_summary.txt"):
        continue

    ticker = filename.split("_")[0]
    
    with open(os.path.join(summaries, filename), "r", encoding="utf-8") as f:
        text = f.read()
    
    # Create Word document
    doc = Document()
    doc.add_heading(f"{ticker} 10-K MD&A Summary", 0)
    doc.add_paragraph(text)

    output_path = os.path.join(output_folder, f"{ticker}_summary.docx")
    doc.save(output_path)
    print(f"Saved {ticker} summary to Word.")